# Windows Server Information #

## Gather Windows Log information ##

**Running Gather Log Events.ps1** as an administrator in PowerShell ISE (if applicable; if not, run in PowerShell as administrator) will collect error messages from the last 30 days found in the Application, Security and/or System Event Logs. 
* The script will export the data to a CSV file (“export.csv”), which can be found in the C:\temp folder on the server in which the script is run. 

**Power Options** – High Performance should always be selected.   

* Check to make sure High Performance is the selected Power Option, and that GPO has not made changes. 

Check for possible new anti-virus software on the server. If found, recommend excluding SQL files from the anti-virus scan. 

http://support.microsoft.com/kb/309422 

### SQL Server Information ###
**Server Properties**
1.	Evaluate current SQL Server version. Report if there is a recent SP (<=2016) or CU (>=2017)
  *  Version reference: https://sqlcollaborative.github.io/builds
  *	There is a specific issue with TDE on early versions of 2016 that is critical. Need to make sure we are patched above CU7 2016 or CU4 2016 SP1 if using TDE.
    ** https://support.microsoft.com/en-us/help/4019893/fix-restore-fails-when-you-do-backup-by-using-compression-and-checksum
2.	Review SQL Server Default locations in Server Properties. The default locations must exist and could cause a service pack or CU install to fail if they do not.
3.	SQL Server Log 
 *	Review the SQL Server Error Log using Error Log.sql. Document any new/suspicious activity.
4.	System Configuration Values 
  *	**Configuration Changes History.sql** checks for system configuration changes.

In [ ]:
--error_log.sql
--#TODO: review string filters at bottom.
--Can execute in a multiserver query
--Execute in Grid mode

PRINT ('Error Log Output Window')
use tempdb
go
select SYSDATETIMEOFFSET()
declare @oldestdate as date, @now as datetime2(0)
select @oldestdate = dateadd(week,-5, sysdatetime()), @now = sysdatetime() --Filter the time frame of the logs.

select 'Getting errors since ' + cast(@oldestdate as varchar(30))

--Get list of logs associated with the SQL Server (by default is 7, probably need more!) 
CREATE TABLE #SQLErrorLogList (
    LogNumber INT NOT NULL,
    LogEndDate datetime2(0) NOT NULL,
    LogSize_b BIGINT NOT NULL);
CREATE NONCLUSTERED INDEX IDX_CL_ell on #SQLErrorLogList (LogNumber) INCLUDE (LogEndDate);

INSERT INTO #SQLErrorLogList
EXEC sys.sp_enumerrorlogs;

--error messages in current log
create table #readerrorlog
( LogDate datetime not null
, LogProcessInfo varchar(255) not null 
, [LogMessageText] varchar(1500) not null 
)

CREATE CLUSTERED INDEX IDX_CL_rel on #readerrorlog (LogDate);

declare @lognumber int = 0, @endoflogfiles bit = 0, @maxlognumber int = 0;

select @maxlognumber =   MAX(LogNumber) from #SQLErrorLogList
WHILE (
		((Select LogEndDate from #SQLErrorLogList where @lognumber = LogNumber) > @oldestdate)
		and @lognumber <= @maxlognumber
		) 
BEGIN

	INSERT INTO #readerrorlog 
	EXEC master.dbo.xp_readerrorlog  
	  @lognumber		--current log file
	, 1					--SQL Error Log
	, N''				--search string 1, must be unicode. Leave empty on purpose, as we do filtering later on.
	, N''				--search string 2, must be unicode. Leave empty on purpose, as we do filtering later on.
	, @oldestdate, @now --time filter. Should be @oldestdate < @now
	, N'desc'			--sort
			
	--print 'including lognumber ' + str(@lognumber)

	set @lognumber = @lognumber + 1	
END
GO

CREATE NONCLUSTERED INDEX IDX_NC_rel on #readerrorlog (Logdate desc, [LogMessageText]) INCLUDE( LogProcessInfo)

GO
--order of servers in a multiserver query is not determinant

--Raw error list
select * from #readerrorlog 
where  1=1
and (	
	LogMessageText like '%error%'
or	LogMessageText like '%failure%'
or	LogMessageText like '%failed%'
or	LogMessageText like '%corrupt%'
)
and LogMessageText not like '%without errors%'
and LogMessageText not like '%returned no errors%'
and LogMessageText not like 'Registry startup parameters:%'
and LogMessageText not like '%informational%'
and LogMessageText not like '%found 0 errors%'
order by LogDate desc;

--Aggregate error counts
select LogMessageText, LogProcessInfo, ErrorCount = count(LogDate), MostRecentOccurrence = max(LogDate) 
from #readerrorlog 
where  1=1
and (	
	LogMessageText like '%error%'
or	LogMessageText like '%failure%'
or	LogMessageText like '%failed%'
or	LogMessageText like '%corrupt%'
)
and LogMessageText not like '%without errors%'
and LogMessageText not like '%returned no errors%'
and LogMessageText not like 'Registry startup parameters:%'
and LogMessageText not like '%informational%'
and LogMessageText not like '%found 0 errors%'
group by LogMessageText, LogProcessInfo
order by count(LogDate) desc, max(LogDate) desc;

SELECT Reboots = LogDate FROM #readerrorlog WHERE LogMessageText like 'Registry startup parameters:%'
ORDER BY LogDate desc;
GO

drop table #readerrorlog
drop table #SQLErrorLogList

In [ ]:
--Configuration Changes History.sql
--Based on the configuration changes history report in SSMS
PRINT('Configuration Change History Output Window')
exec sp_executesql @stmt=N'begin try
declare @enable int;
select @enable = convert(int,value_in_use) from sys.configurations where name = ''default trace enabled''
if @enable = 1 --default trace is enabled
begin
        declare @d1 datetime;
        declare @diff int;  
        declare @curr_tracefilename varchar(500); 
        declare @base_tracefilename varchar(500); 
        declare @indx int ;
        declare @temp_trace table (
                textdata nvarchar(MAX) collate database_default 
        ,       login_name sysname collate database_default
        ,       start_time datetime
        ,       event_class int
        );
        
        select @curr_tracefilename = path from sys.traces where is_default = 1 ; 
        
        set @curr_tracefilename = reverse(@curr_tracefilename)
        select @indx  = PATINDEX(''%\%'', @curr_tracefilename) 
        set @curr_tracefilename = reverse(@curr_tracefilename)
        set @base_tracefilename = LEFT( @curr_tracefilename,len(@curr_tracefilename) - @indx) + ''\log.trc'';
        
        insert into @temp_trace
        select TextData
        ,       LoginName
        ,       StartTime
        ,       EventClass 
        from ::fn_trace_gettable( @base_tracefilename, default ) 
        where ((EventClass = 22 and Error = 15457) or (EventClass = 116 and TextData like ''%TRACEO%(%''))
        
        select @d1 = min(start_time) from @temp_trace
        
        --set @diff= datediff(hh,@d1,getdate())
        --set @diff=@diff/24; 

        select --(row_number() over (order by start_time desc))%2 as l1
                @d1 as TraceStartDate
        ,       start_time as EventDate
		,       case event_class 
                        when 116 then ''Trace Flag '' + substring(textdata,patindex(''%(%'',textdata),len(textdata) - patindex(''%(%'',textdata) + 1) 
                        when 22 then substring(textdata,58,patindex(''%changed from%'',textdata)-60) 
                end as config_option
        ,       login_name
        ,       case event_class 
                        when 116 then ''--''
                        when 22 then substring(substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))
                                                                ,patindex(''%changed from%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata)))+13
                                                                ,patindex(''%to%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - patindex(''%from%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - 6) 
                end as old_value
        ,       case event_class 
                        when 116 then substring(textdata,patindex(''%TRACE%'',textdata)+5,patindex(''%(%'',textdata) - patindex(''%TRACE%'',textdata)-5)
                        when 22 then substring(substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))
                                                                ,patindex(''%to%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata)))+3
                                                                , patindex(''%. Run%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - patindex(''%to%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - 3) 
                end as new_value
        from @temp_trace 
        order by start_time desc
end else 
begin 
        select top 0  1  as l1, 1 as difference,1 as date , 1 as config_option,1 as start_time , 1 as login_name, 1 as old_value, 1 as new_value
end
end try 
begin catch
select  ERROR_NUMBER() as Error_Number
,       ERROR_SEVERITY() as date 
,       ERROR_STATE() as config_option
,       1 as start_time 
,       ERROR_MESSAGE() as login_name
,       1 as old_value, 1 as new_value
end catch',@params=N''

## Performance ##
1.	Run **Page Life Expectancy.sql**
  * Does SQL have enough memory, based on PLE, Churn, and the Target vs Total?
2.	Review CPU Utilization.Sql script and look for any recent +90% periods.
3.	Are there any recent Memory Dumps?
  *	Running **Find Memory Mini Dumps.sql** will let you know if SQL is having issues or has had any  “mini dump crashes” in the recent past.

In [ ]:
--Page Life Expectancy.sql
PRINT('Page Life Expectancy')
select 
	p.InstanceName
,	c.Version 
,	'LogicalCPUCount'		= os.cpu_count
,	OS_Physical_Mem_MB = os.[Server Physical Mem (MB)] -- SQL2012+ only
,	Min_Server_Mem_MB = c.[Min_Server_Mem_MB]
,	Max_Server_Mem_MB = c.[Max_Server_Mem_MB] --2147483647 means unlimited, just like it shows in SSMS
,	p.PLE_s --300s is only an arbitrary rule for smaller memory servers (<16gb), for larger, it should be baselined and measured.
,	'Churn (MB/s)'			=	cast((p.Total_Server_Mem_GB)/1024./NULLIF(p.PLE_s,0) as decimal(19,2))
,	OS_Available_physical_mem_GB = (SELECT cast(available_physical_memory_kb / 1024. / 1024. as decimal(19,2)) from sys.dm_os_sys_memory) 
,	SQL_Physical_memory_in_use_GB = (SELECT cast(physical_memory_in_use_kb / 1024. / 1024. as decimal(19,2)) from sys.dm_os_process_memory)
,	p.Total_Server_Mem_GB --May be more or less than memory_in_use 
,	p.Target_Server_Mem_GB	
,	Target_vs_Total = CASE WHEN p.Total_Server_Mem_GB < p.Target_Server_Mem_GB	 
							THEN 'Target >= Total. SQL wants more memory than it has, or is building up to that point.'
							ELSE 'Total >= Target. SQL has enough memory to do what it wants.' END
,	si.LPIM -- Works on SQL 2016 SP1, 2012 SP4+
from(
select 
	InstanceName = @@SERVERNAME 
,	Target_Server_Mem_GB =	max(case counter_name when 'Target Server Memory (KB)' then convert(decimal(19,3), cntr_value/1024./1024.) end)
,	Total_Server_Mem_GB	=	max(case counter_name when  'Total Server Memory (KB)' then convert(decimal(19,3), cntr_value/1024./1024.) end) 
,	PLE_s	=	max(case counter_name when 'Page life expectancy'  then cntr_value end) 
--select * 
from sys.dm_os_performance_counters
--This only looks at one NUMA node. https://www.sqlskills.com/blogs/paul/page-life-expectancy-isnt-what-you-think/
)  as p
inner join (select 'InstanceName' = @@SERVERNAME, Version = @@VERSION, 
			Min_Server_Mem_MB  = max(case when name = 'min server memory (MB)' then convert(bigint, value_in_use) end) ,
			Max_Server_Mem_MB = max(case when name = 'max server memory (MB)' then convert(bigint, value_in_use) end) 
			from sys.configurations) as c on p.InstanceName = c.InstanceName
inner join (SELECT 'InstanceName' = @@SERVERNAME 
			, cpu_count , hyperthread_ratio AS 'HyperthreadRatio',
			cpu_count/hyperthread_ratio AS 'PhysicalCPUCount'
			, 'Server Physical Mem (MB)' = cast(physical_memory_kb/1024. as decimal(19,2))   -- SQL2012+ only
			FROM sys.dm_os_sys_info ) as os
on c.InstanceName=os.InstanceName


-- Works on SQL 2016 SP1, 2012 SP4+
cross apply (select LPIM = CASE sql_memory_model_Desc 
					WHEN  'Conventional' THEN 'Lock Pages in Memory privilege is not granted'
					WHEN 'LOCK_PAGES' THEN 'Lock Pages in Memory privilege is granted'
					WHEN 'LARGE_PAGES' THEN 'Lock Pages in Memory privilege is granted in Enterprise mode with Trace Flag 834 ON'
					END from sys.dm_os_sys_info 
				) as si

--adapted from http://www.datavail.com/category-blog/max-server-memory-300-second-rule/


In [ ]:
--CPU Utilization.sql
--This is simple use of the ring_buffer for historical CPU, goes back a little over 4 hours.
-- for more CPU and Memory, look at toolbox/sys_dm_os_ring_buffers.sql
PRINT('CPU Utilization Output Window')
select
	Avg_SystemIdle_Pct				=	AVG( record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'int') )
,	Avg_SQLProcessUtilization_Pct	=	AVG( record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'int') )
      from (
            select timestamp, convert(xml, record) as record
            from sys.dm_os_ring_buffers
            where ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
            and record like '%<SystemHealth>%') as x

declare @ts_now bigint
--select @ts_now = cpu_ticks / convert(float, cpu_ticks_in_ms) from sys.dm_os_sys_info
select @ts_now = cpu_ticks / (cpu_ticks/ms_ticks) from sys.dm_os_sys_info;
select	record_id
	,	EventTime				=  dateadd(ms, -1 * (@ts_now - [timestamp]), GetDate()) 
	,	SQLProcessUtilization
	,	SystemIdle
	,	OtherProcessUtilization	= 100 - SystemIdle - SQLProcessUtilization 
from (
      select
            record_id				=	record.value('(./Record/@id)[1]', 'int')
        ,	SystemIdle				=	record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'int') 
        ,	SQLProcessUtilization	=	record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'int') 
        ,	timestamp
      from (
            select timestamp, convert(xml, record) as record
            from sys.dm_os_ring_buffers
            where ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
            and record like '%<SystemHealth>%') as x
      ) as y
order by record_id desc


--http://sqlblog.com/blogs/ben_nevarez/archive/2009/07/26/getting-cpu-utilization-data-from-sql-server.aspx

In [ ]:
--Find Memory Mini Dumps.sql 
-- Get information on location, time and size of any memory dumps from SQL Server  
-- Only SQL 2008R2+
PRINT('Fine Memory Mini Dump Output Window')
SELECT [filename], creation_time, size_in_bytes/1048576.0 AS [Size (MB)]
FROM sys.dm_server_memory_dumps 
ORDER BY creation_time DESC OPTION (RECOMPILE);